<a href="https://colab.research.google.com/github/Rafiul-Git/Relative-Elevation-Model/blob/main/REM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xarray-spatial
!pip install datashader
!pip install py3dep
!pip install pynhd
from __future__ import annotations

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import opt_einsum as oe
import rasterio
import shapely
import xarray as xr
import xrspatial as xs
from datashader import transfer_functions as tf
from datashader import utils as ds_utils
from datashader.colors import Greys9, inferno
from scipy.spatial import KDTree
from shapely import ops

import py3dep
import pygeoutils as geoutils
import pynhd

In [ ]:
bbox = (-110.621713,43.750439,-110.573660,43.781138)
dem_res = py3dep.check_3dep_availability(bbox)
dem_res

In [ ]:
res = 1
dem = py3dep.get_dem(bbox, res)

fig, ax = plt.subplots(figsize=(8, 4), dpi=100)
_ = dem.plot(ax=ax, robust=True)

In [ ]:
wd = pynhd.WaterData("nhdflowline_network")

flw = wd.bybox(bbox)
flw = pynhd.prepare_nhdplus(flw, 0, 0, 0, remove_isolated=True)
flw = flw[flw.levelpathi == flw.levelpathi.min()].to_crs(dem.rio.crs).copy()

fig, ax = plt.subplots(figsize=(8, 4), dpi=100)
flw.plot(ax=ax, color="r")
_ = dem.plot(ax=ax, robust=True)

In [ ]:
river_line = ops.linemerge(flw.geometry.tolist())
npts = int(np.ceil(river_line.length / 10))
river_line = geoutils.smooth_linestring(river_line, 0.1, npts)

url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/USGS_Seamless_DEM_13.vrt"
with rasterio.open(url) as src:
    xy = shapely.get_coordinates(geoutils.geo_transform(river_line, flw.crs, src.crs))
    z = np.ravel(list(geoutils.sample_window(src, xy)))
    river_elev = np.c_[shapely.get_coordinates(river_line), z]

distances = shapely.line_locate_point(river_line, shapely.points(river_line.coords))
plt.figure(figsize=(8, 3), dpi=1000)
_ = plt.plot(distances, river_elev[:, 2])

In [ ]:
from shapely import ops
import rasterio
from scipy.spatial import KDTree

wd = pynhd.WaterData("nhdflowline_network")
flw = wd.bybox(bbox)
flw = pynhd.prepare_nhdplus(flw, 0, 0, 0, remove_isolated=True)
flw = flw[flw.levelpathi == flw.levelpathi.min()].to_crs(dem.rio.crs).copy()

river_line = ops.linemerge(flw.geometry.tolist())
npts = int(np.ceil(river_line.length / 10))
river_line = geoutils.smooth_linestring(river_line, 0.1, npts)

url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/USGS_Seamless_DEM_13.vrt"
with rasterio.open(url) as src:
    xy = shapely.get_coordinates(geoutils.geo_transform(river_line, flw.crs, src.crs))
    z = np.ravel(list(geoutils.sample_window(src, xy)))
    river_elev = np.c_[shapely.get_coordinates(river_line), z]

distances, idxs = KDTree(river_elev[:, :2]).query(
    np.dstack(np.meshgrid(dem.x, dem.y)).reshape(-1, 2),
    k=200,
    workers=-1,
)

w = np.reciprocal(np.power(distances, 2) + np.isclose(distances, 0))
w_sum = np.sum(w, axis=1)
w_norm = oe.contract("ij,i->ij", w, np.reciprocal(w_sum + np.isclose(w_sum, 0)), optimize="optimal")
elevation = oe.contract("ij,ij->i", w_norm, river_elev[idxs, 2], optimize="optimal")
elevation = elevation.reshape((dem.sizes["y"], dem.sizes["x"]))
elevation = xr.DataArray(elevation, dims=("y", "x"), coords={"x": dem.x, "y": dem.y})
rem = dem - elevation

fig, ax = plt.subplots(figsize=(8, 4), dpi=100)
elevation.plot(ax=ax)
_ = flw.plot(ax=ax, color="red")
illuminated = xs.hillshade(dem, angle_altitude=10, azimuth=90)
tf.Image.border = 0
img = tf.stack(
    tf.shade(dem, cmap=Greys9, how="linear"),
    tf.shade(illuminated, cmap=["black", "white"], how="linear", alpha=180),
    tf.shade(rem, cmap=inferno[::-1], span=[0, 7], how="log", alpha=200),
)
ds_utils.export_image(img[::-1], Path("_static", "rem").as_posix())

In [ ]:
from pathlib import Path

# Create the directory
Path("_static").mkdir(exist_ok=True)

# Now save the image
ds_utils.export_image(img[::-1], Path("_static", "rem").as_posix())